## Prioritization for data cleaning

- We should focus on cleaning up the metrics first and do some feature engineering on these to make sure they are in a state that's valuable to FuF
  - Sidewalk damage: HARDSCAPE
  - Size of trees: DBH
  - Density of trees: Lat/Long - create additional metric
  - Neighborhood: PROPERTY
  - Health of tree: CONDITION - definitely needs cleaning, could simplify categories
  - Types of tree: (Genus and Species): BOTANICAL - may be useful to seperate out genus and species here; Common name: COMMON

## Known issues / areas to work on

- ON_ADR/ONSTREET and PROP_ADR/PROPSTREET are the same except for in some cases, we think it may be because of street corners?
- ON_ADR/ONSTREET and PROP_ADR/PROPSTREET need to be merged to produce addresses
- BOTANICAL needs to be split apart into GENUS and Species
- Many categorical variables, especially the metrics, have redundant/messy categorical values that need to be cleaned up.
- We can create binary indicators from many of these categorical variables and aggregate them to create proportions at the neighborhood level, which would be much easier to visualize on the map, and could also then be exported for decision-making purposes.


Look at condition store by species and neighborhood, and look at condition disparity 

In [3]:
import pandas as pd, numpy as np

fuf_data_updated = pd.read_excel('../data/combined_tree_data_with_header_correct_neighboorhoods.xlsx', sheetname=0)

In [4]:
fuf_data_updated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65499 entries, 0 to 65498
Data columns (total 20 columns):
cartodb_id                   65499 non-null int64
status                       55734 non-null float64
trunks                       65499 non-null int64
parkway_largest_dimension    65499 non-null int64
diameter_at_breast_height    65499 non-null int64
tree_id                      55734 non-null float64
clearance                    49865 non-null object
observation_notes            58896 non-null object
hardscape_damage             60687 non-null object
maintenance_notes            65498 non-null object
parkway_space_type           65471 non-null object
condition                    65119 non-null object
common_species_name          65495 non-null object
scientific_species_name      65498 non-null object
side                         65411 non-null object
on_street_name               65498 non-null object
neighborhood                 65499 non-null object
latitude                  

In [ ]:
fuf_data_updated.scientific_species_name.

In [7]:
fuf_data_updated.condition.value_counts()

Fair                   26007
Poor                   16753
Good                   12901
Vacancy                 7503
Dead                     653
Very Good                486
Stump                    197
Stump Removal            191
Excellent                108
Open                     106
Critical                 106
Very                      87
Unsuitable Site           20
2016-06-15 00:00:00        1
dtype: int64

In [8]:
fuf_data_updated[(fuf_data_updated.condition.str.contains('Vacancy')==True)]

,cartodb_id,status,trunks,parkway_largest_dimension,diameter_at_breast_height,tree_id,clearance,observation_notes,hardscape_damage,maintenance_notes,parkway_space_type,condition,common_species_name,scientific_species_name,side,on_street_name,neighborhood,latitude,longitude,correct_neighborhood
84,89477,1,0,6,0,473,NaN,None,None,Infill Plant,Median,Vacancy,Vacant Planting Site - Small,Vacant Planting Site - Small,Median,POINT LOBOS AVE,Seacliff,37.779957,-122.508014,Seacliff
85,89479,1,0,6,0,476,NaN,None,None,Infill Plant,Median,Vacancy,Vacant Planting Site - Small,Vacant Planting Site - Small,Median,POINT LOBOS AVE,Seacliff,37.779947,-122.508168,Seacliff
86,89480,1,0,6,0,475,NaN,None,None,Infill Plant,Median,Vacancy,Vacant Planting Site - Small,Vacant Planting Site - Small,Median,POINT LOBOS AVE,Seacliff,37.779954,-122.508080,Seacliff
102,89709,1,0,13,0,1624,NaN,None,None,Infill Plant,Parkway,Vacancy,Vacant Planting Site - Medium,Vacant Planting Site - Medium,Front,48TH AVE,Seacliff,37.775327,-122.509173,Seacliff
103,89741,1,0,10,0,1623,NaN,None,None,Infill Plant,Parkway,Vacancy,Vacant Planting Site - Medium,Vacant Planting Site - Medium,Front,48TH AVE,Seacliff,37.775365,-122.509181,Seacliff
104,93308,1,0,4,0,820,NaN,None,None,Infill Plant,Well/Pit,Vacancy,Vacant Planting Site_Sawcut,Sawcut New Tree Well Site,Side,CLEMENT ST,Seacliff,37.781082,-122.503738,Seacliff
105,93316,1,0,4,0,822,NaN,None,None,Infill Plant,Well/Pit,Vacancy,Vacant Planting Site_Sawcut,Sawcut New Tree Well Site,Side,CLEMENT ST,Seacliff,37.781074,-122.503881,Seacliff
106,93333,1,0,4,0,823,NaN,None,None,Infill Plant,Well/Pit,Vacancy,Vacant Planting Site_Sawcut,Sawcut New Tree Well Site,Side,CLEMENT ST,Seacliff,37.781074,-122.503952,Seacliff
107,93412,1,0,4,0,793,NaN,None,None,Infill Plant,Well/Pit,Vacancy,Vacant Planting Site_Sawcut,Sawcut New Tree Well Site,Front,CLEMENT ST,Seacliff,37.781021,-122.504929,Seacliff
108,93413,1,0,4,0,794,NaN,None,None,Infill Plant,Well/Pit,Vacancy,Vacant Planting Site_Sawcut,Sawcut New Tree Well Site,Front,CLEMENT ST,Seacliff,37.781021,-122.504996,Seacliff


In [9]:
fuf_data_updated.describe()

,cartodb_id,status,trunks,parkway_largest_dimension,diameter_at_breast_height,tree_id,latitude,longitude
count,65499.000000,55734.000000,65499.000000,65499.000000,65499.000000,55734.000000,65499.000000,65499.000000
mean,52055.193835,1.162396,1.064245,6.621872,8.479687,553945.918416,37.758327,-122.441507
std,31803.990825,0.541284,0.770891,16.043219,10.168778,324443.050568,0.024054,0.029083
min,1.000000,1.000000,0.000000,0.000000,0.000000,12.000000,37.708306,-122.511005
25%,23352.500000,1.000000,1.000000,3.000000,4.000000,301176.250000,37.736114,-122.461827
50%,50009.000000,1.000000,1.000000,3.000000,7.000000,708064.500000,37.761146,-122.441501
75%,86093.500000,1.000000,1.000000,4.000000,12.000000,816154.750000,37.778950,-122.419867
max,111575.000000,6.000000,27.000000,99.000000,1920.000000,912744.000000,37.806489,-122.378641


In [10]:
fuf_data_updated['hardscape_damage'].describe()

count     60687
unique       11
top        None
freq      38118
Name: hardscape_damage, dtype: object

In [11]:
fuf_data_updated['hardscape_damage'].value_counts()

None                  38118
No                     7310
Sidewalk/CG            6256
Sidewalk               5697
Yes                    2344
Curb/Gutter             806
Temporary                95
Well Grate / Cover       25
Other                    22
Private                  13
0                         1
dtype: int64

In [17]:
fuf_data_updated[fuf_data_updated['correct_neighborhood'] != fuf_data_updated['neighborhood']]

,cartodb_id,status,trunks,parkway_largest_dimension,diameter_at_breast_height,tree_id,clearance,observation_notes,hardscape_damage,maintenance_notes,parkway_space_type,condition,common_species_name,scientific_species_name,side,on_street_name,neighborhood,latitude,longitude,correct_neighborhood


In [16]:
fuf_data_updated['correct_neighborhood'].value_counts()

Inner Richmond           7509
Outer Richmond           7425
West of Twin Peaks       6593
Potrero Hill             5387
Bernal Heights           5242
Castro/Upper Market      5146
Outer Mission            4892
Excelsior                3925
Haight Ashbury           3636
Presidio Heights         2833
Pacific Heights          2351
Glen Park                2245
Bayview                  1839
Nob Hill                 1609
Visitacion Valley         959
North Beach               909
Crocker Amazon            711
South of Market           684
Russian Hill              504
Diamond Heights           298
Marina                    295
Downtown/Civic Center     275
Chinatown                 101
Outer Sunset               51
Seacliff                   48
Western Addition           19
Mission                    13
dtype: int64

In [7]:
fuf_data_updated['condition'].value_counts()

Fair               28211
Poor               23342
Fair               19532
Good               12634
Good               11061
Vacancy             8666
Poor                1877
Dead                 741
Very Good            738
Very                 708
Dead                 369
Stump                313
Stump Removal        222
Excellent            175
Critical             120
Open                 106
Critical              85
Unsuitable Site       35
6/15/16                1
dtype: int64

In [8]:
fuf_data_updated['diameter_at_breast_height'].value_counts()

3       8553
4       8544
0       8387
5       8024
2       7681
6       7185
1       6888
7       6732
8       6444
9       5216
10      4997
11      4535
12      4443
13      3853
14      3501
15      2970
16      2560
17      1895
18      1582
19      1191
20       981
21       871
22       699
23       634
24       552
25       447
26       332
27       292
28       246
30       206
        ... 
63         9
80         9
85         9
71         8
64         8
70         7
62         7
73         7
67         6
68         6
65         5
77         5
72         5
87         4
69         4
66         4
90         4
75         4
81         4
74         3
79         3
76         3
78         2
95         2
58         2
88         1
94         1
105        1
1920       1
151        1
dtype: int64

In [31]:
fuf_data_updated['condition'].value_counts()

Fair               28211
Poor               23342
Fair               19532
Good               12634
Good               11061
Vacancy             8666
Poor                1877
Dead                 741
Very Good            738
Very                 708
Dead                 369
Stump                313
Stump Removal        222
Excellent            175
Critical             120
Open                 106
Critical              85
Unsuitable Site       35
6/15/16                1
dtype: int64

In [20]:
# Dead, Vacancy, Poor, Stump, Stump Removal, Unsuitable Site

fuf_data_updated['hardscape_metric'] = fuf_data_updated['hardscape_damage'].apply(lambda x: 0 if x in ['None',
                                                                                    'No', 'NA'] else 1)
fuf_data_updated['vacant_lot_metric'] = fuf_data_updated['condition'].apply(lambda x: 0 if x != 'Vacancy' else 1)
fuf_data_updated['condition_metric'] = fuf_data_updated['condition'].apply(lambda x: 0 if x in ['Poor','Dead',
                                                                               'Critical','Stump','Stump Removal',
                                                                                'Unsuitable Site', 'Open'] else 1)
fuf_data_updated['stump_metric'] = fuf_data_updated['condition'].apply(lambda x: 1 if x in ['Stump', 
                                                                                            'Stump Removal'] else 0)
grouped = fuf_data_updated[['hardscape_metric','diameter_at_breast_height','neighborhood',
          'vacant_lot_metric', 'condition_metric', 'stump_metric']].groupby('neighborhood').mean().reset_index()
joined = pd.merge(fuf_data_updated, grouped, on='neighborhood', how = 'left', suffixes=('_binary','_perc'))

In [21]:
grouped

,neighborhood,hardscape_metric,diameter_at_breast_height,vacant_lot_metric,condition_metric,stump_metric
0,Bayview,0.262643,6.650897,0.000000,0.968461,0.000000
1,Bernal Heights,0.446395,8.650324,0.046547,0.612362,0.006105
2,Castro/Upper Market,0.409639,9.908278,0.021376,0.395453,0.011465
3,Chinatown,0.198020,8.683168,0.000000,0.930693,0.000000
4,Crocker Amazon,0.669480,6.135021,0.180028,0.696203,0.009845
5,Diamond Heights,0.077181,8.624161,0.057047,0.244966,0.006711
6,Downtown/Civic Center,0.094545,7.378182,0.000000,0.952727,0.000000
7,Excelsior,0.563057,7.416306,0.045860,0.623439,0.007134
8,Glen Park,0.457461,10.710022,0.047216,0.484633,0.016927
9,Haight Ashbury,0.342959,9.489549,0.013201,0.908416,0.000275


- There are some neighborhoods that still dont have much data - would it make sense to actually exclude those because of sparsity issues? Those metrics are going to be much less reliable/useful

In [11]:
grouped_with_count = fuf_data_updated[['hardscape_metric','diameter_at_breast_height','neighborhood',
          'vacant_lot_metric', 'condition_metric', 'stump_metric']].groupby('neighborhood').agg(['mean',
                                                                                                'count']).reset_index()
grouped_with_count

neighborhood hardscape_metric        \
                                               mean count   
0                     Alamo Square         0.314371   668   
1                       Anza Vista         0.237991   458   
2                 Bayview District         0.275460  1630   
3                  Bayview Heights         0.163158   190   
4                   Bernal Heights         0.455955  5029   
5             Bernal Heights North         0.379747    79   
6             Bernal Heights South         0.381818    55   
7                 Buena Vista Park         0.289855    69   
8              Castro/Upper Market         0.427029  4817   
9                 Central Richmond         0.205036   556   
10                  Central Sunset         0.123233  5518   
11                      Cow Hollow         0.366972   218   
12                  Crocker Amazon         0.669944   712   
13                 Diamond Heights         0.069536   302   
14           Downtown - Tenderloin         0.144231   104   
15                 Duboce Triangle         0.114286   140   
16  Eureka Valley - Dolores Height         0.212291   179   
17                       Excelsior         0.583444  3781   
18        Financial District North         0.145161   124   
19        Financial District South         0.150943   159   
20                       Glen Park         0.461885  2217   
21                  Haight Ashbury         0.320532  1354   
22                  Haight-Ashbury         0.390167  1261   
23                    Hayes Valley         0.220944  1652   
24                   Hunters Point         0.181818    22   
25                       Ingleside         0.038462    52   
26               Ingleside Heights         0.283582    67   
27               Ingleside Terrace         0.066667    30   
28                   Inner Mission         0.218107   486   
29                  Inner Richmond         0.094110  6248   
..                             ...              ...   ...   
39                 Mission Terrace         0.245614   114   
40            Mount Davidson Manor         0.142857    28   
41                        Nob Hill         0.582647  1579   
42                      Noe Valley         0.362019  5508   
43                     North Beach         0.349705   509   
44                 North Panhandle         0.265625  1472   
45                North Waterfront         0.067797    59   
46                       Oceanview         0.196429    56   
47                   Outer Mission         0.315211  4819   
48                  Outer Parkside         0.076777  3152   
49                  Outer Richmond         0.072946  7156   
50                    Outer Sunset         0.166580  3854   
51                 Pacific Heights         0.977441  2438   
52                        Parkside         0.097125  6435   
53     Parnassus - Ashbury Heights         0.375000    32   
54                  Pine Lake Park         0.136986   146   
55                         Portola         0.026316   114   
56                    Potrero Hill         0.105253  5425   
57                Presidio Heights         0.168990  2367   
58                    Russian Hill         0.563239  1099   
59                     South Beach         0.176471    34   
60                 South of Market         0.125000   568   
61                  Telegraph Hill         0.433333   420   
62                      Twin Peaks         0.141026    78   
63         Van Ness - Civic Center         0.142424   330   
64               Visitacion Valley         0.887841   954   
65                     West Portal         0.014925   134   
66              West of Twin Peaks         0.161024  6409   
67                Western Addition         0.333128  3248   
68                   Westwood Park         0.000000    45   

   diameter_at_breast_height       vacant_lot_metric       condition_metric  \
                        mean count              mean count             mean   
0                  1

In [12]:
joined.columns

Index([u'tree_id', u'neighborhood', u'on_street_name', u'side',
       u'scientific_species_name', u'common_species_name',
       u'diameter_at_breast_height_binary', u'condition',
       u'parkway_space_type', u'parkway_largest_dimension', u'trunks',
       u'maintenance_notes', u'status', u'hardscape_damage',
       u'observation_notes', u'clearance', u'longitude', u'latitude',
       u'hardscape_metric_binary', u'vacant_lot_metric_binary',
       u'condition_metric_binary', u'stump_metric_binary',
       u'hardscape_metric_perc', u'diameter_at_breast_height_perc',
       u'vacant_lot_metric_perc', u'condition_metric_perc',
       u'stump_metric_perc'],
      dtype='object')

In [13]:
joined.head()

,tree_id,neighborhood,on_street_name,side,scientific_species_name,common_species_name,diameter_at_breast_height_binary,condition,parkway_space_type,parkway_largest_dimension,...,latitude,hardscape_metric_binary,vacant_lot_metric_binary,condition_metric_binary,stump_metric_binary,hardscape_metric_perc,diameter_at_breast_height_perc,vacant_lot_metric_perc,condition_metric_perc,stump_metric_perc
0,200610,Presidio Heights,GEARY BLVD,Front,Tristaniopsis laurina,Water Gum,3,Good,Well/Pit,3,...,37.781515,0,0,1,0,0.16899,10.103929,0.084073,0.718631,0.00676
1,200611,Presidio Heights,GEARY BLVD,Front,Stump,Stump,7,Stump,Well/Pit,2,...,37.781501,0,0,1,1,0.16899,10.103929,0.084073,0.718631,0.00676
2,200612,Presidio Heights,GEARY BLVD,Front,Prunus x blireiana,Double-Flowering Plum,6,Fair,Well/Pit,2,...,37.781497,0,0,1,0,0.16899,10.103929,0.084073,0.718631,0.00676
3,200614,Presidio Heights,GEARY BLVD,Front,Metrosideros excelsa,New Zealand Christmas Tree,11,Fair,Well/Pit,4,...,37.781487,0,0,1,0,0.16899,10.103929,0.084073,0.718631,0.00676
4,200615,Presidio Heights,GEARY BLVD,Front,Metrosideros excelsa,New Zealand Christmas Tree,12,Good,Well/Pit,4,...,37.781484,0,0,1,0,0.16899,10.103929,0.084073,0.718631,0.00676


In [14]:
joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112331 entries, 0 to 112330
Data columns (total 27 columns):
tree_id                             112331 non-null object
neighborhood                        112331 non-null object
on_street_name                      112320 non-null object
side                                107443 non-null object
scientific_species_name             112330 non-null object
common_species_name                 112279 non-null object
diameter_at_breast_height_binary    112331 non-null int64
condition                           108936 non-null object
parkway_space_type                  112302 non-null object
parkway_largest_dimension           112331 non-null int64
trunks                              112331 non-null int64
maintenance_notes                   112330 non-null object
status                              75304 non-null float64
hardscape_damage                    106101 non-null object
observation_notes                   100767 non-null object
clearan

In [15]:
joined.to_csv("fuf_with_metrics.csv")